### Scenario C
Last updated Mar 11, 2022

In [ ]:
from sys import path as pylib
import os
pylib += [os.path.abspath('../')]

In [ ]:
import pickle
from transfer_learning.transfer_utils import get_device
from models.trainers import train_with_grid_search, train_with_CI
from models.models_manager import BasicMLPModelManager
from configs.features_sets import ALL_FEATURES
from configs.training import BIN_CLASSIFICATION_WEIGHTS, MLP_EVAL_METRIC
from utils.utility_functions import get_best_params
from configs.files import *

In [ ]:
device = get_device()

## 1. DNN

### 1.1 DNN (Inductive Transfer) - grid search

In [ ]:
model_manager = BasicMLPModelManager(from_pretrained=True, device=device,
                                     pretrained_path='../transfer_learning/pretrained_models/MLP_Scenario_C.pkl') # Add pre-trained models here
MLP_IT_GS_results, weekly_best_params = train_with_grid_search(model_manager=model_manager, interval=7,
                                                               train_data_path=PREVENT_EXP1_PATH,
                                                               features=ALL_FEATURES, eval_metric=MLP_EVAL_METRIC,
                                                               test_data_path=TEST_EXP1_PATH,
                                                               class_weights=BIN_CLASSIFICATION_WEIGHTS)
MLP_IT_GS_results.to_csv(EXP_3_MLP_IT_GS_FILE, index=False)
with open(EXP_3_MLP_IT_WEEKLY_PARAMS_FILE, 'wb') as f:
    pickle.dump(weekly_best_params, f)

### 1.2 DNN (Inductive Transfer) - Confidence interval estimation

In [ ]:
model_manager = BasicMLPModelManager(from_pretrained=True, device=device,
                                     pretrained_path='../transfer_learning/pretrained_models/MLP_Scenario_C.pkl') # Add pre-trained models here

with open(EXP_3_MLP_IT_WEEKLY_PARAMS_FILE, 'rb') as f:
    best_mlp_params = pickle.load(f)

params = get_best_params(params_AUC_pairs=best_mlp_params, n_weeks=3)
MLP_TL_CI_results = train_with_CI(model_manager=model_manager, train_data_path=PREVENT_EXP1_PATH, features=ALL_FEATURES,
                                  test_data_path=TEST_EXP1_PATH, interval=7, params=params,
                                  eval_metric=MLP_EVAL_METRIC, class_weights=BIN_CLASSIFICATION_WEIGHTS)
MLP_TL_CI_results.to_csv(EXP_3_MLP_IT_CI_FILE, index=False)